# Tutorial 1: Agents

## Functionalities:
- Task-based Agents which will break down tasks into subtasks and solve them in bite-sized portions
- Agents with registered functions as skills

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Agent Basics
- Create an agent by entering your agent's name and description
- Agents are task-based, so they will help generate subtasks to fulfil your main task

- Agents are made to be non-verbose, so they will just focus only on task instruction (Much more efficient compared to conversational-based agentic frameworks like AutoGen)
- Agent's interactions will be stored into `subtasks_completed` by default, which will serve as a memory buffer for future interactions

- **Inputs for Agent**:
    - **agent_name**: String. Name of agent, hinting at what the agent does
    - **agent_description**: String. Short description of what the agent does
    - **max_subtasks**: Int. Default: 5. The maximum number of subtasks the agent can have
    - **verbose**: Bool. Default: True. Whether to print out agent's intermediate thoughts
<br/><br/>

- **Agent Internal Parameters**:
    - **Task**: String. The task the agent has been assigned to - Defaults to "No task assigned"
    - **Subtasks Completed**: Dict. The keys are the subtask names and the values are the result of the respective subtask
    - **Is Task Completed**: Bool. Whether the current Task is completed
<br/><br/>

- **Automatic Task Running**
    - **reset()**: Resets the Agent Internal Parameters
    - **run(task: str)**: Performs the task. Do note that agent's state will not be reset, so if you want to reset it, call reset() prior to running this
<br/><br/>


- **Manual Task Running**
    - **assign(task: str)**: Assigns a task to the agent, setting Is Task Completed to False
    - **run(num_subtasks: int)**: Runs the task for num_subtasks number of subtasks. Uses the existing assigned task
<br/><br/>

- **Give User Output**
    - **reply_user(query: str = '', stateful: bool = True)**: Using all information from subtasks, give a reply about the `query` to the user. If `query` is not given, then it replies based on the current task the agent is doing. If `stateful` is True, saves this query and reply into `subtasks_completed`
<br/><br/>
    
- **Check status of Agent**:
    - **status()**: Lists out Agent Name, Agent Description, Available Functions (default function is to use the LLM), Task, Subtasks Completed and Is Task Completed
    
## Example Agent Creation
```python
my_agent = Agent('Helpful assistant', 'You are a generalist agent')
```

## Example Agent Task Running - Split the assigned task into subtasks and execute each of them

```python
output = my_agent.run('Give me 5 words rhyming with cool, and make a 4-sentence poem using them')
```

`Subtask identified: Find 5 words that rhyme with 'cool'`

`Getting LLM to perform the following task: Find 5 words that rhyme with 'cool'`
> pool, rule, fool, tool, school

`Subtask identified: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`

`Getting LLM to perform the following task: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`
> In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool.

`Task completed successfully!`

## Example Agent Reply to User - Reference the subtasks' output to answer the user's query
```python
output = my_agent.reply_user()
```

`
Here are 5 words that rhyme with "cool": pool, rule, fool, tool, school. Here is a 4-sentence poem using these words: "In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool."
`

## Check Agent's Status
```python
my_agent.status()
```

`Agent Name: Helpful assistant`

`Agent Description: You are a generalist agent`

`Available Functions: ['use_llm', 'end_task']`

`Task: Give me 5 words rhyming with cool, and make a 4-sentence poem using them`

`Subtasks Completed:`

`Subtask: Find 5 words that rhyme with 'cool'`

`pool, rule, fool, tool, school`

`Subtask: Compose a 4-sentence poem using the words 'pool', 'rule', 'fool', 'tool', and 'school'`

`In the school, the golden rule is to never be a fool. Use your mind as a tool, and always follow the pool.`

`Is Task Completed: True`

## Create Agent

In [3]:
# Create your agent by specifying name and description
my_agent = Agent('Helpful assistant', 'You are a generalist agent')

In [4]:
# Show the agent status - By default agent comes equipped with default function `use_llm` which queries the llm
# end_task is to end the current task if it is completed
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


## Run Task! (Example 1)

In [5]:
# Do the task by subtasks. This does generation to fulfil task
my_agent.reset()
output = my_agent.run('Give me 5 words rhyming with cool, and then make a 4-sentence poem using them')

Observation: No subtasks have been completed yet.
Thoughts: To complete the assigned task, I need to find 5 words rhyming with "cool" and then create a 4-sentence poem using them.
Subtask identified: Find 5 words rhyming with "cool".
Getting LLM to perform the following task: Find 5 words rhyming with "cool".
> Here are 5 words that rhyme with "cool": drool, pool, stool, school, fool.

Observation: The user requested 5 words rhyming with "cool" and a 4-sentence poem using them. I have already found 5 words that rhyme with "cool": drool, pool, stool, school, fool.
Thoughts: To complete the remainder of the assigned task, I need to create a 4-sentence poem using the words drool, pool, stool, school, and fool.
Subtask identified: Create a 4-sentence poem using the words drool, pool, stool, school, and fool.
Getting LLM to perform the following task: Create a 4-sentence poem using the words drool, pool, stool, school, and fool.
> In a school by the pool, sat a fool on a stool, making other

In [6]:
# visualise the subtask outputs of the task
print(output)

[{'Detailed Outcome': 'Here are 5 words that rhyme with "cool": drool, pool, stool, school, fool.'}, {'Detailed Outcome': 'In a school by the pool, sat a fool on a stool, making others drool. His antics were cool, breaking every rule.'}]


In [7]:
# Generates a meaningful reply to the user about the task according to its current state. Functions like a QA bot
output = my_agent.reply_user()

In a school by the pool, sat a fool on a stool, making others drool. His antics were cool, breaking every rule.


In [8]:
# You can also ask your questions to the agent, and the agent will reply according to its current state. Functions like a QA bot
# Here, we set stateful is false because we do not want this to go into subtasks_completed
output = my_agent.reply_user('Where is the pool?', stateful = False)

The pool is located by the school where the fool sat on a stool.


In [9]:
# see the updated agent status
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: Give me 5 words rhyming with cool, and then make a 4-sentence poem using them
Subtasks Completed:
Subtask: Find 5 words rhyming with "cool".
{'Detailed Outcome': 'Here are 5 words that rhyme with "cool": drool, pool, stool, school, fool.'}

Subtask: Create a 4-sentence poem using the words drool, pool, stool, school, and fool.
{'Detailed Outcome': 'In a school by the pool, sat a fool on a stool, making others drool. His antics were cool, breaking every rule.'}

Subtask: Give me 5 words rhyming with cool, and then make a 4-sentence poem using them
In a school by the pool, sat a fool on a stool, making others drool. His antics were cool, breaking every rule.

Is Task Completed: True


In [10]:
# when you change the task, we will set the task completed to False so you can do it again. 
# Do note that the earlier subtasks_history might affect your new task and should be reset if it is not relevant
my_agent.reset()
output = my_agent.run('Give me 5 words rhyming with task, and then make a 4-sentence poem using them')

Observation: No subtasks completed yet.
Thoughts: Need to provide 5 words rhyming with "task" and then create a 4-sentence poem using them.
Subtask identified: Generate 5 words rhyming with "task".
Getting LLM to perform the following task: Generate 5 words rhyming with "task".
> Here are 5 words that rhyme with "task": flask, mask, bask, cask, and ask.

Observation: The user requested 5 words rhyming with "task" and a 4-sentence poem using them. The 5 rhyming words have been provided: flask, mask, bask, cask, and ask.
Thoughts: To complete the remainder of the Assigned Task, a 4-sentence poem using the rhyming words needs to be created.
Subtask identified: Create a 4-sentence poem using the words: flask, mask, bask, cask, and ask.
Getting LLM to perform the following task: Create a 4-sentence poem using the words: flask, mask, bask, cask, and ask.
> In a sunlit meadow, a flask I did find, hidden beneath a golden mask so kind. I took a moment to bask in the warmth of the day, beside a 

In [11]:
# see the updated agent staus
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: Give me 5 words rhyming with task, and then make a 4-sentence poem using them
Subtasks Completed:
Subtask: Generate 5 words rhyming with "task".
{'Detailed Outcome': 'Here are 5 words that rhyme with "task": flask, mask, bask, cask, and ask.'}

Subtask: Create a 4-sentence poem using the words: flask, mask, bask, cask, and ask.
{'Detailed Outcome': "In a sunlit meadow, a flask I did find, hidden beneath a golden mask so kind. I took a moment to bask in the warmth of the day, beside a wooden cask where memories lay. And as I pondered life's questions, I had but one thing to ask - what more could one need in this peaceful flask?"}

Is Task Completed: True


## Run Task! (Example 2)

In [12]:
# Create your agent by specifying name and description
my_agent = Agent('Trip Planner',
'''You are a planner for a trip.
When given a destination, first do an overall broad plan for all days with no mention of the time 
e.g. Day 1 - Going to City for Activities, Day 2 - Going to the Wild for Activities

Thereafter, fill in details for the morning, afternoon and evening activities, 
e.g. Day 1 Morning - Shopping, Afternoon - Eating, Evening - Theatre

Do not do anything other than generating the plan''')

In [13]:
# Do the task by subtasks
output = my_agent.run('Give me a broad plan and a detailed plan for a 3-day itinerary for Hawaii, involving scuba-diving and some sightseeing')

Observation: No subtasks have been completed yet.
Thoughts: I need to start by creating a broad plan and a detailed plan for a 3-day itinerary for Hawaii, involving scuba-diving and some sightseeing.
Subtask identified: Generate a broad plan for a 3-day itinerary for Hawaii, involving scuba-diving and some sightseeing.
Getting LLM to perform the following task: Generate a broad plan for a 3-day itinerary for Hawaii, involving scuba-diving and some sightseeing.
> {
    Day 1 - Exploring the underwater world with scuba diving
    Morning - Scuba diving at Shipwreck Beach
    Afternoon - Visit to Waimea Canyon State Park
    Evening - Relaxing at Poipu Beach

    Day 2 - More scuba diving adventures and sightseeing
    Morning - Scuba diving at Tunnels Beach
    Afternoon - Explore the Na Pali Coast
    Evening - Enjoy a traditional Hawaiian Luau

    Day 3 - Final day of exploration
    Morning - Snorkeling at Hanauma Bay
    Afternoon - Visit Pearl Harbor and the USS Arizona Memorial
  

In [14]:
# Generates a meaningful reply to the user
output = my_agent.reply_user()

{
    Day 1 - Exploring the underwater world with scuba diving
    Morning - Scuba diving at Shipwreck Beach
    Afternoon - Visit to Waimea Canyon State Park
    Evening - Relaxing at Poipu Beach

    Day 2 - More scuba diving adventures and sightseeing
    Morning - Scuba diving at Tunnels Beach
    Afternoon - Explore the Na Pali Coast
    Evening - Enjoy a traditional Hawaiian Luau

    Day 3 - Final day of exploration
    Morning - Snorkeling at Hanauma Bay
    Afternoon - Visit Pearl Harbor and the USS Arizona Memorial
    Evening - Farewell dinner at a local seafood restaurant
}


In [15]:
my_agent.status()

Agent Name: Trip Planner
Agent Description: You are a planner for a trip.
When given a destination, first do an overall broad plan for all days with no mention of the time 
e.g. Day 1 - Going to City for Activities, Day 2 - Going to the Wild for Activities

Thereafter, fill in details for the morning, afternoon and evening activities, 
e.g. Day 1 Morning - Shopping, Afternoon - Eating, Evening - Theatre

Do not do anything other than generating the plan
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: Give me a broad plan and a detailed plan for a 3-day itinerary for Hawaii, involving scuba-diving and some sightseeing
Subtasks Completed:
Subtask: Generate a broad plan for a 3-day itinerary for Hawaii, involving scuba-diving and some sightseeing.
{'Detailed Outcome': '{\n    Day 1 - Exploring the underwater world with scuba diving\n    Morning - Scuba diving at Shipwreck Beach\n    Afternoon - Visit to Waimea Canyon State Park\n    Evening - Relaxing at Poi

## Manual Stepping Through each Subtask

In [16]:
# Create your agent by specifying name and description
my_agent = Agent('Number Expert', 'You are great with numbers')

In [17]:
# Assign a task to the agent
my_agent.assign_task('List me three random numbers from 1 to 50, then give me their sum')

In [18]:
# Subtask completed and Is Task Completed should be reset
my_agent.status()

Agent Name: Number Expert
Agent Description: You are great with numbers
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: List me three random numbers from 1 to 50, then give me their sum
Subtasks Completed: None
Is Task Completed: False


In [19]:
output = my_agent.run(num_subtasks = 1)

Observation: No subtasks have been completed yet
Thoughts: To list three random numbers from 1 to 50 and then find their sum, I can use a general function to generate the random numbers and calculate the sum
Subtask identified: Generate three random numbers from 1 to 50 and calculate their sum
Getting LLM to perform the following task: Generate three random numbers from 1 to 50 and calculate their sum
> As a Number Expert, I have generated three random numbers: 12, 28, and 45. The sum of these numbers is 85.



In [20]:
# visualise the outputs of the task at the subtask level
print(output)

[{'Detailed Outcome': 'As a Number Expert, I have generated three random numbers: 12, 28, and 45. The sum of these numbers is 85.'}]


In [21]:
# Visualise the first subtask
my_agent.status()

Agent Name: Number Expert
Agent Description: You are great with numbers
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: List me three random numbers from 1 to 50, then give me their sum
Subtasks Completed:
Subtask: Generate three random numbers from 1 to 50 and calculate their sum
{'Detailed Outcome': 'As a Number Expert, I have generated three random numbers: 12, 28, and 45. The sum of these numbers is 85.'}

Is Task Completed: False


In [22]:
output = my_agent.run(num_subtasks = 1)

Observation: I have generated three random numbers: 12, 28, and 45. The sum of these numbers is 85.
Thoughts: The remaining part of the Assigned Task is to provide the sum of the three random numbers. This can be achieved by using the end_task function to pass the final output to the user.
Subtask identified: End Task
Task completed successfully!



In [23]:
# visualise the outputs of the task at the subtask level
print(output)

[{'Detailed Outcome': 'As a Number Expert, I have generated three random numbers: 12, 28, and 45. The sum of these numbers is 85.'}]


In [24]:
# Visualise the first and second subtasks
my_agent.status()

Agent Name: Number Expert
Agent Description: You are great with numbers
Available Functions: ['use_llm', 'end_task']
Shared Variables: ['agent']
Task: List me three random numbers from 1 to 50, then give me their sum
Subtasks Completed:
Subtask: Generate three random numbers from 1 to 50 and calculate their sum
{'Detailed Outcome': 'As a Number Expert, I have generated three random numbers: 12, 28, and 45. The sum of these numbers is 85.'}

Is Task Completed: True


In [25]:
# if task already completed, then no need to do further
output = my_agent.run(num_subtasks = 1)

Task already completed!

Subtasks completed:
Subtask: Generate three random numbers from 1 to 50 and calculate their sum
{'Detailed Outcome': 'As a Number Expert, I have generated three random numbers: 12, 28, and 45. The sum of these numbers is 85.'}



In [26]:
# Generates a meaningful reply to the user
output = my_agent.reply_user()

As a Number Expert, I have generated three random numbers: 12, 28, and 45. The sum of these numbers is 85.


# Functions
- Enhances ```strict_json()``` with a function-like interface for repeated use of modular LLM-based functions (or wraps external functions)
- Use angle brackets <> to enclose input variable names. First input variable name to appear in `fn_description` will be first input variable and second to appear will be second input variable. For example, `fn_description = 'Adds up two numbers, <var1> and <var2>'` will result in a function with first input variable `var1` and second input variable `var2`
- (Optional) If you would like greater specificity in your function's input, you can describe the variable after the : in the input variable name, e.g. `<var1: an integer from 10 to 30>`. Here, `var1` is the input variable and `an integer from 10 to 30` is the description.
- (Optional) If your description of the variable is one of `int`, `float`, `str`, `dict`, `list`, `array`, `Dict[]`, `List[]`, `Array[]`, `Enum[]`, `bool`, we will enforce type checking when generating the function inputs in `get_next_subtask` method of the `Agent` class. Example: `<var1: int>` Refer to Tutorial 0, Section 3. Type Forcing Output Variables for details.
- Inputs (primary):
    - **fn_description**: String. Function description to describe process of transforming input variables to output variables. Variables must be enclosed in <> and listed in order of appearance in function input.
        - New feature: If `external_fn` is provided and no `fn_description` is provided, then we will automatically parse out the fn_description based on docstring of `external_fn`. Only requirement is that the docstring must contain the names of all compulsory input variables
    - **output_format**: Dict. Dictionary containing output variables names and description for each variable.
    
- Inputs (optional):
    - **examples** - Dict or List[Dict]. Examples in Dictionary form with the input and output variables (list if more than one)
    - **external_fn** - Python Function. If defined, instead of using LLM to process the function, we will run the external function. 
        If there are multiple outputs of this function, we will map it to the keys of `output_format` in a one-to-one fashion
    - **fn_name** - String. If provided, this will be the name of the function. Ohterwise, if `external_fn` is provided, it will be the name of `external_fn`. Otherwise, we will use LLM to generate a function name from the `fn_description`
    - **kwargs** - Dict. Additional arguments you would like to pass on to the strict_json function
        
- Outputs:
    JSON of output variables in a dictionary (similar to ```strict_json```)
    
#### Example Internal LLM-Based Function
```python
# Construct the function: var1 will be first input variable, var2 will be second input variable and so on
sentence_style = Function(fn_description = 'Output a sentence with words <var1> and <var2> in the style of <var3>', 
                     output_format = {'output': 'sentence'})

# Use the function
sentence_style('ball', 'dog', 'happy') #var1, var2, var3
```

#### Example Output
```{'output': 'The happy dog chased the ball.'}```
    
#### Example External Function
```python
def binary_to_decimal(x):
    return int(str(x), 2)

# an external function with a single output variable, with an expressive variable description
b2d = Function(fn_description = 'Convert input <x: a binary number in base 2> to base 10', 
            output_format = {'output1': 'x in base 10'},
            external_fn = binary_to_decimal)

# Use the function
b2d(10) #x
```

#### Example Output
```{'output1': 2}```

#### Example fn_description and output_format inferred from type hints and docstring of External Function
```python
# Docstring must provide all input variables
# We will ignore shared_variables, *args and **kwargs
from typing import List
def add_number_to_list(num1: int, num_list: List[int], *args, **kwargs) -> List[int]:
    '''Adds num1 to num_list'''
    num_list.append(num1)
    return num_list

fn = Function(external_fn = add_number_to_list, 
    #output_format = {'num_array': 'Array of numbers'} ## If you would like to name output variables (helps with LLM understanding), define your own output_format
             )

str(fn)
```

#### Example Output
`Description: Adds Adds <num1: int> to <num_list: list[int]>`

`Input: ['num1', 'num_list']`

`Output: {'output_1': 'list[int]'}`

In [27]:
# Example Internal Function
sentence_style = Function(fn_description = 'Output a sentence with <obj> and <entity> in the style of <emotion>', 
                     output_format = {'output': 'sentence'}, fn_name = 'sentence_with_objects_entities_emotion')

In [28]:
# Example External Function
def binary_to_decimal(x):
    return int(str(x), 2)

# a single-valued external function, with more expressive variable description
b2d = Function(fn_description = 'Convert input <x: a binary number in base 2> to base 10', 
            output_format = {'output1': 'x in base 10'},
            external_fn = binary_to_decimal)

In [29]:
# Docstring must provide all input variables
# We will ignore shared_variables, *args and **kwargs
from typing import List
def add_number_to_list(num1: int, num_list: List[int], *args, **kwargs) -> List[int]:
    '''Adds num1 to num_list'''
    num_list.append(num1)
    return num_list

fn = Function(external_fn = add_number_to_list, 
    #output_format = {'num_array': 'Array of numbers'} ## If you would like to name output variables (helps with LLM understanding), define your own output_format
             )

str(fn)

"Description: Adds <num1: int> to <num_list: list[int]>\nInput: ['num1', 'num_list']\nOutput: {'output_1': 'list[int]'}\n"

# Power Up your Agents - Bring in Functions (aka Tools)
- After creating your agent, use `assign_functions` to assign a list of functions (of class Function) to it
- Function names will be automatically inferred if not specified
- Proceed to run tasks by using `run()`

```python
my_agent = Agent('Helpful assistant', 'You are a generalist agent')

my_agent.assign_functions([sentence_style, b2d])

output = my_agent.run('Generate me a happy sentence with a number and a ball. The number is 1001 converted to decimal')
```

`Subtask identified: Convert the binary number 1001 to decimal`
`Calling function binary_to_decimal with parameters {'x': '1001'}`

> {'output1': 9}

`Subtask identified: Generate a happy sentence with the decimal number and a ball`
`Calling function sentence_with_objects_entities_emotion with parameters {'obj': '9', 'entity': 'ball', 'emotion': 'happy'}`

> {'output': 'I am so happy with my 9 balls.'}

`Task completed successfully!`

- Approach 1: Automatically Run your agent using `run()`

- Approach 2: Manually select and use functions for your task
    - **select_function(task: str)**: Based on the task, output the next function name and input parameters
    - **use_function(function_name: str, function_params: dict, subtask: str = '', stateful: bool = True)**: Uses the function named `function_name` with `function_params`. `stateful` controls whether the output of this function will be saved to `subtasks_completed` under the key of `subtask`
<br/><br/>

- **Assign/Remove Functions**:
    - **assign_functions(function_list: list)**: Assigns a list of functions to the agent
    - **remove_function(function_name: str)**: Removes function named function_name from the list of assigned functions
<br/><br/>

- **Show Functions**:
    - **list_functions()**: Returns the list of functions of the agent
    - **print_functions()**: Prints the list of functions of the agent
<br/><br/>

In [30]:
# Create your agent
my_agent = Agent(agent_name = 'Helpful assistant', 
      agent_description = 'You are a generalist agent')

In [31]:
# Assign functions
my_agent.assign_functions(function_list = [b2d, sentence_style])

In [32]:
# Show the functions the agent has
my_agent.print_functions()

Name: use_llm
Description: For general tasks. Used only when no other function can do the task
Input: []
Output: {'Output': 'Output of LLM'}

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}

Name: binary_to_decimal
Description: Convert input <x: a binary number in base 2> to base 10
Input: ['x']
Output: {'output1': 'x in base 10'}

Name: sentence_with_objects_entities_emotion
Description: Output a sentence with <obj> and <entity> in the style of <emotion>
Input: ['obj', 'entity', 'emotion']
Output: {'output': 'sentence'}



## Automatic Usage of Functions

In [33]:
# multi-task with multiple functions
output = my_agent.run('Generate me a happy sentence with a number (convert from binary: 1001) and a ball')

Observation: No subtasks have been completed yet.
Thoughts: To complete the assigned task, I need to generate a happy sentence with a number (convert from binary: 1001) and a ball.
Subtask identified: Convert the binary number 1001 to base 10
Calling function binary_to_decimal with parameters {'x': 1001}
> {'output1': 9}

Observation: Subtask Completed: Convert the binary number 1001 to base 10: 9
Thoughts: The remaining part of the assigned task is to generate a happy sentence with the number 9 (converted from binary 1001) and a ball.
Subtask identified: Output a sentence with "9" and "ball" in the style of "happy"
Calling function sentence_with_objects_entities_emotion with parameters {'obj': 9, 'entity': 'ball', 'emotion': 'happy'}
> {'output': 'I am so happy to have 9 balls!'}

Observation: The binary number 1001 has been converted to base 10, which is 9. A happy sentence has been generated with "9" and "ball" in the style of "happy".
Thoughts: The next and final subtask is to end 

In [34]:
# visualise the subtask outputs of the task
print(output)

[{'output1': 9}, {'output': 'I am so happy to have 9 balls!'}]


In [35]:
# give a response to user
output = my_agent.reply_user()

I am so happy to have 9 balls!


# Manual Usage of Functions
- If you want to just see what the agent would choose for a hypothetical task, use `select_function`. This will not update the internal state, and will output function name and function params
- If you want specificity in the process, you can just execute the agent's functions yourself using `use_function`
- `use_function`: Uses an agent's function using name and params. Note that by default there will be updating of subtasks_completed when performing the function. In order not to update subtasks_completed, set `stateful = False`

In [36]:
# this should call generate_sentence_with_emotion
my_agent.reset()
function_name, function_params = my_agent.select_function(
    task = 'Output a sentence with bell, dog and happy')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

my_agent.use_function(function_name, function_params, stateful = False)

Observation: The assigned task is to output a sentence with bell, dog and happy.
Thoughts: To complete the assigned task, I need to generate a sentence with the given objects and emotion.
Selected function name: sentence_with_objects_entities_emotion
Selected function params: {'obj': 'bell', 'entity': 'dog', 'emotion': 'happy'}
Calling function sentence_with_objects_entities_emotion with parameters {'obj': 'bell', 'entity': 'dog', 'emotion': 'happy'}
> {'output': 'The dog wagged its tail happily when it heard the bell.'}



{'output': 'The dog wagged its tail happily when it heard the bell.'}

In [37]:
# this should call binary_to_decimal
my_agent.reset()
function_name, function_params = my_agent.select_function(
    task = 'What is the decimal representation of binary number 101?')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

my_agent.use_function(function_name, function_params, stateful = False)

Observation: The Assigned Task is to find the decimal representation of the binary number 101.
Thoughts: The next step is to convert the binary number 101 to its decimal representation.
Selected function name: binary_to_decimal
Selected function params: {'x': 101}
Calling function binary_to_decimal with parameters {'x': 101}
> {'output1': 5}



{'output1': 5}

In [38]:
# this should call use_llm
my_agent.reset()
function_name, function_params = my_agent.select_function(
    task = 'Research on the benefits of exercise')
print(f'Selected function name: {function_name}\nSelected function params: {function_params}')

my_agent.use_function(function_name, function_params, stateful = False)

Observation: No subtasks completed for Assigned Task
Thoughts: Need to start working on the Assigned Task
Selected function name: use_llm
Selected function params: {'instruction': 'Research on the benefits of exercise'}
Getting LLM to perform the following task: Research on the benefits of exercise
> Exercise has numerous benefits for both physical and mental health. Regular physical activity can help prevent chronic diseases such as heart disease, diabetes, and certain types of cancer. It also improves mood, reduces stress, and enhances cognitive function. Additionally, exercise can help with weight management, improve sleep quality, and boost overall energy levels. It is recommended to engage in a combination of aerobic, strength training, and flexibility exercises for optimal health benefits.



{'Detailed Outcome': 'Exercise has numerous benefits for both physical and mental health. Regular physical activity can help prevent chronic diseases such as heart disease, diabetes, and certain types of cancer. It also improves mood, reduces stress, and enhances cognitive function. Additionally, exercise can help with weight management, improve sleep quality, and boost overall energy levels. It is recommended to engage in a combination of aerobic, strength training, and flexibility exercises for optimal health benefits.'}

# Saving and Loading Agents
Sometimes you want to configure your agents and save them and load them elsewhere, while maintaining the current agent state

- When you use the `save_agent` function, we store the entire agent's internal state, include name, description, list of functions, subtasks history and all other internal variables into a pickle file
- When you use the `load_agent` function, and we will load the entire agent saved in the pickle file into the existing agent

Key functions:
- **save_agent(pickle_file_name: str)**: Saves the agent's internal parameters to a pickle file named pickle_file_name (include .pkl), returns the pickle file
- **load_agent(pickle_file_name: str)**: Loads the agent's internal parameters from a pickle file named pickle_file_name (include .pkl), returns loaded agent

#### Example 1: Saving Agent
```python
my_agent.save_agent('myagent.pkl')
```

#### Example Output
```Agent saved to myagent.pkl```

#### Example 2: Loading Agent
```python
new_agent = Agent().load_agent('myagent.pkl')
```

#### Example Output
```Agent loaded from myagent.pkl```

In [39]:
# run one task to add something to subtasks_completed
my_agent.reset()
output = my_agent.run('What is the decimal representation of binary number 101?')

Observation: No subtasks completed yet.
Thoughts: Need to convert the binary number 101 to decimal representation.
Subtask identified: Convert the binary number 101 to decimal representation.
Calling function binary_to_decimal with parameters {'x': 101}
> {'output1': 5}

Observation: The binary number 101 has been successfully converted to its decimal representation, which is 5
Thoughts: The task is almost complete, only the final step of providing the decimal representation to the user is left
Subtask identified: End Task
Task completed successfully!



In [40]:
# see status of agent before saving
my_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'binary_to_decimal', 'sentence_with_objects_entities_emotion']
Shared Variables: ['agent']
Task: What is the decimal representation of binary number 101?
Subtasks Completed:
Subtask: Convert the binary number 101 to decimal representation.
{'output1': 5}

Is Task Completed: True


In [41]:
# save agent
my_agent.save_agent('myagent.pkl')

Agent saved to myagent.pkl


In [42]:
# load agent using load_agent code
new_agent = Agent().load_agent('myagent.pkl')

Agent loaded from myagent.pkl


In [43]:
# see status of loaded agent (which also includes what is saved in subtasks_completed, and shared_variables)
new_agent.status()

Agent Name: Helpful assistant
Agent Description: You are a generalist agent
Available Functions: ['use_llm', 'end_task', 'binary_to_decimal', 'sentence_with_objects_entities_emotion']
Shared Variables: ['agent']
Task: What is the decimal representation of binary number 101?
Subtasks Completed:
Subtask: Convert the binary number 101 to decimal representation.
{'output1': 5}

Is Task Completed: True


# Contributing agent to community

Once you are done developing your agent, we encourage you to contribute your agent to taskgen community.
Agent contribution works by creating a Pull Request to taskgen repo, hence you need to have a github profile. We have taken efforts to make the process simple

Follow below steps to contribute your agent -
- Create your profile on github
- Setup git on your local laptop
- Set envrionment variable `GITHUB_USERNAME` as your github user name
- Set envrionment variable `GITHUB_TOKEN` as your github token. Steps to create github token below
    - Sign-in to your github account on your browser
    - Navigate to token settings on github - `https://github.com/settings/tokens`
    - Generate new token (classic)
        - Provide token name, expiration
        - Slect scope - repo:public_repo
- Execute `contribute_agent` function on your agent

In [44]:
os.environ['GITHUB_USERNAME'] = 'hardikmaheshwari'
os.environ['GITHUB_TOKEN'] = '<REDACTED>'

In [46]:
my_agent.contribute_agent()

hardikmaheshwari already has a fork of taskgen
Fork already exists as remote. Using existing fork - fork
Pushed changes to fork fork in branch contribute-agent-HelpfulAssistant successfully
Pull request created successfully


'Pull Request created successfully at https://github.com/simbianai/taskgen/pull/22'